### Day 3: Binary Diagnostic

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day03/puzzle_0_2_sample.ipynb)


#### Part Two. Sample data

In [ ]:
using Microsoft.DotNet.Interactive.Formatting;
Formatter.Register<int>(v => Convert.ToString(v,  2).PadLeft(5, '0'))

In [ ]:
#load "common.fsx"
#load "../common.fsx"

In [ ]:
#!value --name diagnosticRaw --from-file ./sample

In [ ]:
#!share --from value diagnosticRaw

let reportSample = parseDiagnostic diagnosticRaw

reportSample

index,value
0,00100
1,11110
2,10110
3,10111
4,10101
5,01111
6,00111
7,11100
8,10000
9,11001


Similarly to Part 1, we are going to check which bit value prevails, but do this only among entries having that value at that position.

Removing a value from a suitable data structure might be beneficial performance-wise, but let's just rely heavily on binary arithmetic and comparison.

We are gonna reuse the function which calculates the most common bit

In [ ]:
let getMostCommon i numbers = 
    let maski = 1 <<< i
    let ``0s`` = numbers |> Seq.filter (fun n -> n &&& maski = 0) |> Seq.length
    let ``1s`` = Array.length numbers - ``0s``
    if (``0s`` > ``1s``) then 0 else maski

For the first position and oxygen generator rating bit `1` prevails:

In [ ]:
let step1 = getMostCommon 4 reportSample
step1

10000

On step 2 we want to look for the most common bit only among values starting from `1`:

In [ ]:
let step2 = 
    reportSample
    |> Array.filter (fun x -> 0b1_0000 = (x &&& 0b1_0000))
    |> displayPipe
    |> getMostCommon 3
step2

index,value
0,11110
1,10110
2,10111
3,10101
4,11100
5,10000
6,11001


00000

Step 3

In [ ]:
let step3 = 
    reportSample
    |> Array.filter (fun x -> 0b10_000 = (x &&& 0b11_000))
    |> displayPipe
    |> getMostCommon 2
step3

index,value
0,10110
1,10111
2,10101
3,10000


00100

Step 4

In [ ]:
let step4 = 
    reportSample
    |> Array.filter (fun x -> 0b101_00 = (x &&& 0b111_00))
    |> displayPipe
    |> getMostCommon 1
step4

index,value
0,10110
1,10111
2,10101


00010

Step 5

In [ ]:
reportSample
|> Array.filter (fun x -> 0b1011_0 = (x &&& 0b1111_0))
|> getMostCommon 0
|> display


00001

By this moment, there is only one value matching the aggregated mask left. This will be the oxygen generator rating.

In [ ]:
let ogr = 
    reportSample
    |> Array.filter (fun x -> 0b10111 = (x &&& 0b11111))
    |> Array.exactlyOne
ogr |> display
ogr |> uint64 |> display

10111

23

So, generally speaking, when iterating over positions, we can aggregate a mask which values will be checked with. The second mask is basically a function from the position.

In [ ]:
type State = { Mask : int; Pos: int }

let rec ogr' state =
    if (state.Pos < 0) then state
    else
        let matching = 
            reportSample
            |> Array.filter (fun x -> state.Mask = (0b11111 <<< (state.Pos+1) &&& x))
        matching
        |> Array.tryExactlyOne 
        |> function 
            | Some value -> { Mask = value; Pos = 0}
            | None -> ogr' { Pos = state.Pos - 1; 
                            Mask = matching |> getMostCommon state.Pos ||| state.Mask}

let ogr = (ogr' {Mask = 0b00000; Pos = 4}).Mask
ogr |> display
ogr |> uint64 |> display

10111

23

For the CO2 scrubber rating, however, we need to choose values differently, by the least common bit

In [ ]:
let getLeastCommon i numbers = 
    let maski = 1 <<< i
    let ``0s`` = numbers |> Seq.filter (fun n -> n &&& maski = 0) |> Seq.length
    let ``1s`` = Array.length numbers - ``0s``
    if (``0s`` <= ``1s``) then 0 else maski

In [ ]:
type State = { Mask : int; Pos: int }

let rec co2sr' state =
    if (state.Pos < 0) then state
    else
        let matching = 
            reportSample
            |> Array.filter (fun x -> state.Mask = (0b11111 <<< (state.Pos+1) &&& x))
        matching
        |> Array.tryExactlyOne 
        |> function 
            | Some value -> { Mask = value; Pos = 0}
            | None -> co2sr' { Pos = state.Pos - 1; 
                            Mask = matching |> getLeastCommon state.Pos ||| state.Mask}

let co2sr = (ogr' {Mask = 0b00000; Pos = 4}).Mask
co2sr |> display
co2sr |> uint64 |> display

10111

23